In [6]:
with open("data_inputs/day16_input.txt") as f:
    input = f.read()

In [73]:
d = """0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111"""

d_l = d.split("\n")
h2b = {dd[0]: dd[4:8] for dd in d_l}


In [74]:
def hex2bin(input):

    output = []
    for _i in input:
        output.append(h2b[_i])

    return "".join(output)


In [75]:
input_bin = hex2bin(input)


In [9]:
class Packet():
    def __init__(self, ):
        self.version = None
        self.type_id = None
        self.length_type = None
        self.length = None
        self.subpackets = [] 
        self.literals = []

class Operator():



SyntaxError: unexpected EOF while parsing (655339795.py, line 11)

In [10]:
# Packet: VVV TTT AAAAA BBBBB CCCCC
# Operator: VVV TTT I L(I(0):x15 length / I(1):x11 subpackets)

In [ ]:
bitf = -1
packets = []
p = 0

while i < len(input_bin):º
    bit0 = bitf + 1

    packets[p] = {"version": int(input_bin[bit0:bit0+3], 2)}

    packets[p]["type_id"] = int(input_bin[bit0+4:bit0+6], 2)

    if packets[p]["type_id"] == 4:
        next_packet(packets[p])

    else:
        next_operator(packets[p])


In [32]:
bitf_literal = 17
bitf_literal if (bitf_literal % 8) == 0 else (bitf_literal + (8 - (bitf_literal % 8)))

24

In [134]:
def next_packet(bit0, versions, input_bin, is_subpacket):

    print(bit0, input_bin[bit0:(bit0+3)])
    versions.append(int(input_bin[bit0:(bit0+3)], 2))

    # Packet
    if input_bin[(bit0+3):(bit0+6)] == '100':
        print("Packet", input_bin[(bit0):(bit0+3)], input_bin[(bit0+3):(bit0+6)])
        
        zero = False
        bit0_literal = bit0+6
        while not zero:
            if input_bin[bit0_literal] == '0':
                zero = True
            else:
                bit0_literal += 5
        
        bitf_literal = bit0_literal + 5

        if is_subpacket:
            print("is_subpacket", input_bin[(bit0+6):(bitf_literal)])
            return bitf_literal

        print("not is_subpacket", input_bin[(bit0+6):(bitf_literal)])
        return bitf_literal if (bitf_literal % 8) == 0 else (bitf_literal + (8 - (bitf_literal % 8)))

    # Operator
    else:
        if input_bin[bit0+6] == '0':
            print("Operator 0", input_bin[bit0:bit0+3], input_bin[bit0+3:bit0+6], input_bin[bit0+6], input_bin[(bit0+7):(bit0+22)])

            subpacket_len = int(input_bin[(bit0+7):(bit0+22)], 2)
            l = 0
            sub_bit0 = bit0+22
            sub_bitf = None

            print("start of len", subpacket_len, ":", sub_bit0)
            while l < subpacket_len:
                sub_bitf = next_packet(sub_bit0, versions, input_bin, True)
                l += sub_bitf - sub_bit0
                sub_bit0 = sub_bitf
            print("end of len:", sub_bitf)    
            
            return sub_bitf if (sub_bitf % 8) == 0 else (sub_bitf + (8 - (sub_bitf % 8)))
        
        else:
            
            print("Operator 1", input_bin[bit0:bit0+3], input_bin[bit0+3:bit0+6], input_bin[bit0+6], input_bin[(bit0+7):(bit0+18)])
            num_subpackets = int(input_bin[(bit0+7):(bit0+18)], 2)
            sub_bit0 = bit0+18
            sub_bitf = None

            for sp in range(num_subpackets):
                sub_bitf = next_packet(sub_bit0, versions, input_bin, True)
                sub_bit0 = sub_bitf

            return sub_bitf if (sub_bitf % 8) == 0 else (sub_bitf + (8 - (sub_bitf % 8)))

In [106]:
_input_bin = hex2bin("620080001611562C8802118E34")
_input_bin

'01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100'

In [86]:
_input_bin = hex2bin("8A004A801A8002F478")
_input_bin

'100010100000000001001010100000000001101010000000000000101111010001111000'

In [131]:
versions

[3, 0, 0, 5, 4, 3]

In [130]:
len(_input_bin)

104

In [129]:
_input_bin[64:]

'1000100000000010000100011000111000110100'

In [128]:
len(_input_bin[:64])

64

In [113]:
int("00000000010", 2)

2

In [124]:
int("000000000010110", 2)

22

In [135]:
versions = []
bitf = 0

while bitf < len(_input_bin)-1:
    
    bitf = next_packet(bitf, versions, _input_bin, False)
            





0 011
Operator 1 011 000 1 00000000010
18 000
Operator 0 000 000 0 000000000010110
start of len 22 : 40
40 000
Packet 000 100
is_subpacket 01010
51 101
Packet 101 100
is_subpacket 01011
end of len: 62
64 100
Operator 0 100 010 0 000000010000100
start of len 132 : 86
86 011
Operator 1 011 000 1 11000110100
104 


ValueError: invalid literal for int() with base 2: ''